In [ ]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")
client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)

In [2]:
# set path for testing
import sys
import os
parent_dir = os.path.abspath("../")
print(parent_dir)
sys.path.append(parent_dir)

/Users/hanna/openfn/ai_experiments/apollo/services


In [92]:
from embeddings import snomed_store, loinc_store

In [ ]:
from embeddings.embeddings import SearchResult

In [340]:
snomed_store = snomed_store.connect_snomed()

In [93]:
loinc_store = loinc_store.connect_loinc()

In [6]:
snomed_store.search("nasal")

/var/folders/y3/tg4ts42x7bl1ty6zww9fx4bh0000gn/T/ipykernel_1923/3112543583.py:122: LangChainDeprecationWarning: The class `Pinecone` was deprecated in LangChain 0.0.3 and will be removed in 0.3.0. Use :class:`~PineconeVectorStore` instead.
  store = self.VectorStoreClass(


[SearchResult(text='{"Value Set Name": "Body Site Value Set", "Description": "Entire mucous membrane of nasal cavity and nasal sinus (body structure)", "Purpose: Clinical Focus": "All SNOMED CT anatomic structures, locations, abnormal structures that can be considered to describe an anatomical site."}', metadata={'Code': 736508000.0, 'Description': 'Entire mucous membrane of nasal cavity and nasal sinus (body structure)', 'Purpose: Clinical Focus': 'All SNOMED CT anatomic structures, locations, abnormal structures that can be considered to describe an anatomical site.', 'Value Set Name': 'Body Site Value Set'}, score=None),
 SearchResult(text='{"Value Set Name": "Body Site Value Set", "Description": "Entire mucous membrane of nasal cavity and nasal sinus (body structure)", "Purpose: Clinical Focus": "All SNOMED CT anatomic structures, locations, abnormal structures that can be considered to describe an anatomical site."}', metadata={'Code': 736508000.0, 'Description': 'Entire mucous me

In [118]:
# import data to temporarily plug in missing metadata that will be in the vector store
import pandas as pd

df = pd.read_csv("/Users/hanna/openfn/ai_experiments/data/LOINC-Clinical-Terminology/LoincTableCore.csv")
loinc_num_dict = dict(zip(df.LONG_COMMON_NAME.to_list(), df.LOINC_NUM.to_list()))

/var/folders/y3/tg4ts42x7bl1ty6zww9fx4bh0000gn/T/ipykernel_4572/1852639281.py:4: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/Users/hanna/openfn/ai_experiments/data/LOINC-Clinical-Terminology/LoincTableCore.csv")


Pipeline for searching for approximate synonyms or other textual similarities

Approach:
Step 1: Use input term to search the vector store.
Step 2: Use an LLM to pick one result from the search results. It will give its reasoning as well which could perhaps be consulted by the user.
Step 3: Use an LLM to extract only the name from the Step 2 answer to select the full entry from the DB & use the correct code.


Problems: 
- Some do not look similar. E.g. avpu_score should map to "Level of responsiveness" in LOINC
    Possible solutions:
    - Add another LLM step before the search. Could selectively add disambiguation.

-The similarity search is biased against the simplest answer (e.g. hits "Self-Reported Maximum Adult Height" etc. not "Body height")
    Possible solutions:
    - check missing entries in db; try different search methods and thresholds/Ks

In [160]:
input_query = "avpu_score"
# input_query = "hepatitis_b_result"
input_query = input_query.replace("_", " ")
retreived_data = loinc_store.search(input_query, search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.01})
retreived_data


[SearchResult(text='{"LONG_COMMON_NAME": "Papova virus SV 40 DNA [#/volume] (viral load) in Urine by NAA with probe detection", "METHOD_TYP": "Probe.amp.tar", "CLASS": "MICRO", "SYSTEM": "Urine"}', metadata={'CLASS': 'MICRO', 'LONG_COMMON_NAME': 'Papova virus SV 40 DNA [#/volume] (viral load) in Urine by NAA with probe detection', 'METHOD_TYP': 'Probe.amp.tar', 'SYSTEM': 'Urine'}, score=None),
 SearchResult(text='{"LONG_COMMON_NAME": "Parvovirus B19 DNA [#/volume] (viral load) in Upper respiratory specimen by NAA with probe detection", "METHOD_TYP": "Probe.amp.tar", "CLASS": "MICRO", "SYSTEM": "Respiratory.upper"}', metadata={'CLASS': 'MICRO', 'LONG_COMMON_NAME': 'Parvovirus B19 DNA [#/volume] (viral load) in Upper respiratory specimen by NAA with probe detection', 'METHOD_TYP': 'Probe.amp.tar', 'SYSTEM': 'Respiratory.upper'}, score=None),
 SearchResult(text='{"LONG_COMMON_NAME": "Papova virus SV 40 DNA [#/volume] (viral load) in Serum or Plasma by NAA with probe detection", "METHOD_TY

In [108]:
retreived_data = loinc_store.search(input_query, search_kwargs={"k": 50})
retreived_data

[SearchResult(text='{"LONG_COMMON_NAME": "Hepatitis B virus e Ag [Presence] in Serum, Plasma or Blood by Rapid immunoassay", "METHOD_TYP": "IA.rapid", "CLASS": "MICRO", "SYSTEM": "Ser/Plas/Bld"}', metadata={'CLASS': 'MICRO', 'LONG_COMMON_NAME': 'Hepatitis B virus e Ag [Presence] in Serum, Plasma or Blood by Rapid immunoassay', 'METHOD_TYP': 'IA.rapid', 'SYSTEM': 'Ser/Plas/Bld'}, score=None),
 SearchResult(text='{"LONG_COMMON_NAME": "Hepatitis B virus surface Ab [Presence] in Serum, Plasma or Blood by Rapid immunoassay", "METHOD_TYP": "IA.rapid", "CLASS": "MICRO", "SYSTEM": "Ser/Plas/Bld"}', metadata={'CLASS': 'MICRO', 'LONG_COMMON_NAME': 'Hepatitis B virus surface Ab [Presence] in Serum, Plasma or Blood by Rapid immunoassay', 'METHOD_TYP': 'IA.rapid', 'SYSTEM': 'Ser/Plas/Bld'}, score=None),
 SearchResult(text='{"LONG_COMMON_NAME": "Hepatitis B virus e Ab [Presence] in Serum, Plasma or Blood by Rapid immunoassay", "METHOD_TYP": "IA.rapid", "CLASS": "MICRO", "SYSTEM": "Ser/Plas/Bld"}', m

In [ ]:
# TODO missing metadata in LOINC - codes
# TODO add missing sections to snomed emebddings
# TODO check all terms are def in embeddings. Spot cheet respiratory rate 9279-1
# TODO maybe pick threshold first, if not enough results use top k

In [326]:
synonym_system_prompt = """
You are an assisstant for matching fields between health record systems. 
You will be given a source text, and a set of candidate target records to pick from. 
Pick only one target record. If there is not enough information, pick the most generic or common one.
"""

synonym_user_prompt = """
Source text to search for: "{input_text}"

Here are the candidate target health record fields:

"{retreived_data}"
"""

In [327]:

def synonym_method_1(input_query):

    retreived_data = loinc_store.search(input_query.replace("_", " "), search_kwargs={"k": 100})

    synonym_user_prompt_formatted = synonym_user_prompt.format(input_text=input_query, retreived_data=retreived_data)

    message = client.messages.create(
        model= "claude-3-5-sonnet-20241022", #"claude-3-5-sonnet-20241022", #claude-3-5-haiku-20241022
        max_tokens=500,
        temperature=0,
        system=synonym_system_prompt,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": synonym_user_prompt_formatted
                    }
                ]
            }
        ]
    )
    answer = message.content[0].text

    return answer

In [328]:
synonym_method_1('hepatitis b result')

'Since the source text "hepatitis b result" is very generic and doesn\'t specify which type of hepatitis B test (surface antigen, antibody, etc.), I\'ll select the most common and basic hepatitis B test, which is the surface antigen (HBsAg) test.\n\nI select:\n{"LONG_COMMON_NAME": "Hepatitis B virus surface Ag [Presence] in Serum or Plasma by Immunoassay", "METHOD_TYP": "IA", "CLASS": "MICRO", "SYSTEM": "Ser/Plas"}\n\nRationale:\n1. Surface antigen (HBsAg) is the primary screening test for hepatitis B infection\n2. This uses standard immunoassay methodology (not rapid or specialized testing)\n3. Can be done on either serum or plasma (most common specimen types)\n4. Tests for presence/absence which is the basic result type\n5. This is the most general purpose and commonly ordered hepatitis B test'

In [329]:
results = []
for field in fields:
    result = synonym_method_1(field)
    results.append([field, result])
results_synonym_1_df = pd.DataFrame(results, columns=["Input field", "System answer"])
results_synonym_1_df.to_csv("results_1220_synonym_method_1.csv", index=False)

Synonym approach 2) Guess & Search

This approach aims to address the problems noted above the first method (when input texts do not look at all like the output). Leverage the LLM's in-built intuition for clinical terminology.

Step 1 - Use LLM to output the LOINC text
Step 2 - Use the text from the LLM output to search for the entry. (option b if we see issues: use both original query + the LLM output to search the db)

In [170]:
synonym_expand_system_prompt = """
You are an assisstant for matching terminology between health record systems. 
You will be given a source text from one health system, which might be in another language.
You should output its equivalent name (the long name, not the code) in LOINC clinical terminology with no further explanation.
"""

synonym_expand_user_prompt = """
Source text to output a LOINC term for: "{input_text}"
"""

In [244]:
synonym_expand_select_system_prompt = """
You are an assisstant for matching terminology between health record systems. 
You will be given a LOINC entry to look for in a list of similar records. You will also be given the original soruce term for context.
You should select the correct LOINC entry from the options and output just the LONG_COMMON_NAME and the LOINC_NUMBER separated by a semicolon.
"""

synonym_expand_select_user_prompt = """
Original source term for context: {input_text}
LOINC entry to look for: {LLM_guess}
LOINC entries to select from: "{retreived_texts}"
"""

In [257]:
# use LLM to get input text CT synonym then search vector store. Also add hybrid search.

def synonym_mapping(input_query):
    # input_query = "malaria_RDT_result"

    synonym_expand_user_prompt_formatted = synonym_expand_user_prompt.format(input_text=input_query)
    message = client.messages.create(
        model= "claude-3-5-sonnet-20241022", #"claude-3-5-sonnet-20241022", #claude-3-5-haiku-20241022
        max_tokens=500,
        temperature=0,
        system=synonym_expand_system_prompt,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": synonym_expand_user_prompt_formatted
                    }
                ]
            }
        ]
    )
    synonym_expand_answer = message.content[0].text
    print(f"LLM guess for the CT: {synonym_expand_answer}")
    synonym_expand_retreived_data = loinc_store.search(input_query, search_kwargs={"k": 100})

    # add keyword search to conduct hybrid search
    keyword_search_results = df[df["LONG_COMMON_NAME"].str.contains(synonym_expand_answer)].LONG_COMMON_NAME.to_list()
    keyword_search_results = [SearchResult(text=json.dumps({"LONG_COMMON_NAME": s}), metadata={}, score=None) for s in keyword_search_results]
    search_results_combined = keyword_search_results+synonym_expand_retreived_data

    # temp fix for missing loinc numbers
    search_results_combined = [SearchResult(text=r.text, metadata={**r.metadata, 'LOINC_NUMBER': loinc_num_dict.get(json.loads(r.text)['LONG_COMMON_NAME'])}, score=r.score) for r in search_results_combined]

    # tried this rule-based retreival, but there are unexpected issues e.g. "Body height" hits "Body height Mother"
    # final_search_result = ""
    # for search_result in synonym_expand_retreived_data:
    #     if synonym_expand_answer in search_result.metadata["LONG_COMMON_NAME"]:
    #         final_search_result = (search_result, loinc_num_dict[search_result.metadata["LONG_COMMON_NAME"]])
    # final_search_result

    synonym_expand_select_user_prompt_formatted = synonym_expand_select_user_prompt.format(input_text=input_query, LLM_guess=synonym_expand_answer, retreived_texts=search_results_combined)
    message = client.messages.create(
        model= "claude-3-5-sonnet-20241022", #"claude-3-5-sonnet-20241022", #claude-3-5-haiku-20241022
        max_tokens=500,
        temperature=0,
        system=synonym_expand_select_system_prompt,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": synonym_expand_select_user_prompt_formatted
                    }
                ]
            }
        ]
    )
    synonym_expand_select_answer = message.content[0].text
    
    return synonym_expand_select_answer

In [367]:
synonym_method_2 = synonym_mapping

In [ ]:
# # remove additional text if LLM included it
# synonym_expand_select_answer = synonym_expand_select_answer.replace('{"LONG_COMMON_NAME": "', '').replace('"}', '')
# final_search_result = ""
# for search_result in synonym_expand_retreived_data:
#     if synonym_expand_select_answer in search_result.metadata["LONG_COMMON_NAME"]:
#         final_search_result = (search_result, loinc_num_dict[search_result.metadata["LONG_COMMON_NAME"]])
# final_search_result

In [253]:
fields = [
   'systolic_blood_pressure_mmhg',
   'diastolic_blood_pressure_mmhg',
   'temperature',
   'pulse_bpm',
   'respiration_rate_rpm',
   'avpu_score',
   'height',
   'weight',
   'head',
   'eyes',
   'ear_nose_throat',
   'has_the_patient_ever_had_a_genital_infection',
   'abdomen',
   'heart',
   'cholesterol_test_results',
   'cervical_cancer_result',
   'hemoglobin_result',
   'hepatitis_b_result',
   'hepatitis_c_result',
   'hiv_result',
   'tuberculosis_test_result',
   'urinalysis_result',
   'urinary_hcg_result',
   'gdpgd2ppgds_result',
   'asam_urat_results',
   'malaria_RDT_result',
   'malaria_RDT_test',
   'igg_result',
   'igm_result',
   'bhcg_result',
   'how_many_pregnancies_have_they_had',
   'how_many_pregnancies_have_they_aborted_or_miscarried',
   'first_day_of_last_menstrual_period',
   'estimated_date_of_delivery',
   'bmi',
   'muac',
   'smoking_history',
   'how_many_alcoholic_drinks_per_week',
   'extremities_arm_leg',
   'musculoskeletal',
   'neurological',
   'skin',
   'lungs',
   'uterine_height_cm',
   'syphilis_result',
   'blood_oxygen_satutation',
   'current_contraceptive',
   'non-prescribed_drug_use',
   'how_many_children',
   'extraoral'
]

In [255]:
gold_labels = [
    '# Pregnancies;11996-6',
    '# Births.still living;11638-4',
    'Birth control method at intake Reported;86649-1',
    'Body height;8302-2',
    'Body mass index (BMI) Ratio;39156-5',
    'Body temperature;8310-5',
    'Body weight;29463-7',
    'Choriogonadotropin.beta subunit (pregnancy test) Presence in Urine;2112-1',
    'Choriogonadotropin.beta subunit Units/volume in Serum or Plasma;21198-7',
    'Cholesterol Mass/volume in Serum or Plasma;2093-3',
    'Delivery date Estimated;11778-8',
    'Diastolic blood pressure;8462-4',
    'Fasting glucose Mass/volume in Serum or Plasma;1558-6',
    'Glucose Mass/volume in Serum or Plasma --2 hours post meal;1521-4',
    'Glucose [Mass/volume] in Capillary blood by Glucometer;41653-7',
    'HIV 1 Ab [Presence] in Serum, Plasma or Blood by Rapid immunoassay;68961-2',
    'Heart rate;8867-4',
    'Hemoglobin Mass/volume in Blood;718-7',
    'Hepatitis B virus surface Ag [Presence] in Serum, Plasma or Blood by Rapid immunoassay;75410-1',
    'Hepatitis C virus Ab Presence in Serum;16128-1',
    'History of Alcohol use;11331-6',
    'I took medication that had not been prescribed or if had been prescribed, I took more than the prescribed dose;72639-8',
    'IgG Mass/volume in Serum or Plasma;2465-3',
    'IgM Mass/volume in Serum or Plasma;3182-3',
    'Last menstrual period start date;8665-2',
    'Level of responsiveness;67775-7',
    'Microscopic observation [Identifier] in Sputum by Acid fast stain;11477-7',
    'Oral evaluation exam;34045-5',
    'Other pregnancy outcomes #;69043-8',
    'Oxygen saturation in Arterial blood;2708-6',
    'Physical findings of Abdomen Narrative;10191-5',
    'Physical findings of Extremities Narrative;10196-4',
    'Physical findings of Heart Narrative;10200-4',
    'Physical findings of Lung;32449-1',
    'Physical findings of Musculoskeletal system Narrative;11410-8',
    'Physical findings of Neurologic balance and Coordination;8712-2',
    'Physical findings of Skin Narrative;10206-1',
    'Plasmodium sp Ag Identifier in Blood by Rapid immunoassay;70569-9',
    'Respiratory rate;9279-1',
    'Systolic blood pressure;8480-6',
    'Tobacco smoking status;72166-2',
    'Treponema pallidum Ab [Presence] in Serum by Hemagglutination;8041-6',
    'Urate Mass/volume in Serum or Plasma;3084-1',
    'Urinalysis complete panel - Urine;24356-8',
    'Uterus Fundal height Tape measure;11881-0'
]

In [ ]:
l

In [314]:
results_df

,Input field,Correct answer,System answer
0,systolic_blood_pressure_mmhg,# Pregnancies;11996-6,Systolic blood pressure;8480-6
1,diastolic_blood_pressure_mmhg,# Births.still living;11638-4,None of the provided LOINC entries exactly mat...
2,temperature,Birth control method at intake Reported;86649-1,Body temperature;8310-5
3,pulse_bpm,Body height;8302-2,"Based on the source term ""pulse_bpm"" and looki..."
4,respiration_rate_rpm,Body mass index (BMI) Ratio;39156-5,Respiratory rate;9279-1
5,avpu_score,Body temperature;8310-5,"I don't see the exact LOINC entry ""Level of co..."
6,height,Body weight;29463-7,Body height;8302-2
7,weight,Choriogonadotropin.beta subunit (pregnancy tes...,Body weight;29463-7
8,head,Choriogonadotropin.beta subunit Units/volume i...,Physical findings of Head;8701-5
9,eyes,Cholesterol Mass/volume in Serum or Plasma;2093-3,Physical findings of Eye;8699-1


In [ ]:
results = []
for field, gold_label in zip(fields, gold_labels):
    result = synonym_mapping(field)
    results.append([field, gold_label, result])
results_df = pd.DataFrame(results, columns=["Input field", "Correct answer", "System answer"])
results_df.to_csv("results_1220.csv", index=False)

LLM guess for the CT: Systolic blood pressure
LLM guess for the CT: Diastolic blood pressure by Automated oscillometric method
LLM guess for the CT: Body temperature
LLM guess for the CT: Heart rate
LLM guess for the CT: Respiratory rate
LLM guess for the CT: Level of consciousness by AVPU scale
LLM guess for the CT: Body height
LLM guess for the CT: Body weight
LLM guess for the CT: Body region Head
LLM guess for the CT: Eyes - Examination
LLM guess for the CT: Ears, nose, throat, and larynx
LLM guess for the CT: History of Genital infection
LLM guess for the CT: Body region Abdomen
LLM guess for the CT: Heart structure
LLM guess for the CT: Cholesterol [Mass/volume] in Serum or Plasma
LLM guess for the CT: Cervical cancer screening results
LLM guess for the CT: Hemoglobin [Mass/volume] in Blood
LLM guess for the CT: Hepatitis B virus surface Ag [Presence] in Serum or Plasma
LLM guess for the CT: Hepatitis C virus RNA [Presence] in Serum or Plasma by NAA with probe detection
LLM guess

/var/folders/y3/tg4ts42x7bl1ty6zww9fx4bh0000gn/T/ipykernel_4572/2735813380.py:29: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  keyword_search_results = df[df["LONG_COMMON_NAME"].str.contains(synonym_expand_answer)].LONG_COMMON_NAME.to_list()


LLM guess for the CT: Glucose-6-phosphate dehydrogenase [Enzymatic activity/volume] in Red Blood Cells
LLM guess for the CT: Uric acid [Mass/volume] in Serum or Plasma
LLM guess for the CT: Malaria rapid diagnostic test
LLM guess for the CT: Malaria rapid diagnostic test
LLM guess for the CT: Immunoglobulin G [Mass/volume] in Serum or Plasma
LLM guess for the CT: Immunoglobulin M [Mass/volume] in Serum or Plasma
LLM guess for the CT: Beta HCG [Mass/volume] in Serum or Plasma
LLM guess for the CT: Pregnancies - total number
LLM guess for the CT: Number of abortions and miscarriages
LLM guess for the CT: Date of last menstrual period
LLM guess for the CT: Delivery date Estimated from last menstrual period
LLM guess for the CT: Body mass index
LLM guess for the CT: Mid-upper arm circumference
LLM guess for the CT: Tobacco smoking status
LLM guess for the CT: Alcoholic drinks per week [#]
LLM guess for the CT: Extremities examination
LLM guess for the CT: Body structure of musculoskeletal 

In [319]:
# add missing
missing = [
    'gdp',
    'gds_result',
]
results = []
for field in missing:
    result = synonym_mapping(field)
    results.append([field, result])
results_missing_df = pd.DataFrame(results, columns=["Input field", "System answer"])
results_missing_df

LLM guess for the CT: Glucose-6-phosphate dehydrogenase [Enzymatic activity/volume] in Blood
LLM guess for the CT: Geriatric Depression Scale total score


,Input field,System answer
0,gdp,None of the provided LOINC entries match the t...
1,gds_result,Geriatric depression scale (GDS) total;48544-1


In [320]:
results_missing_df["System answer"].to_list()

['None of the provided LOINC entries match the target entry "Glucose-6-phosphate dehydrogenase [Enzymatic activity/volume] in Blood". I cannot provide a LONG_COMMON_NAME and LOINC_NUMBER since the correct match is not present in the given list.',
 'Geriatric depression scale (GDS) total;48544-1']

In [259]:
results_df

,Input field,Correct answer,System answer
0,systolic_blood_pressure_mmhg,# Pregnancies;11996-6,Systolic blood pressure;8480-6
1,diastolic_blood_pressure_mmhg,# Births.still living;11638-4,None of the provided LOINC entries exactly mat...
2,temperature,Birth control method at intake Reported;86649-1,Body temperature;8310-5
3,pulse_bpm,Body height;8302-2,"Based on the source term ""pulse_bpm"" and looki..."
4,respiration_rate_rpm,Body mass index (BMI) Ratio;39156-5,Respiratory rate;9279-1
5,avpu_score,Body temperature;8310-5,"I don't see the exact LOINC entry ""Level of co..."
6,height,Body weight;29463-7,Body height;8302-2
7,weight,Choriogonadotropin.beta subunit (pregnancy tes...,Body weight;29463-7
8,head,Choriogonadotropin.beta subunit Units/volume i...,Physical findings of Head;8701-5
9,eyes,Cholesterol Mass/volume in Serum or Plasma;2093-3,Physical findings of Eye;8699-1


Approach 3) Add full dataset in prompt

',LOINC_NUM,COMPONENT,PROPERTY,TIME_ASPCT,SYSTEM,SCALE_TYP,METHOD_TYP,CLASS,CLASSTYPE,LONG_COMMON_NAME,SHORTNAME,EXTERNAL_COPYRIGHT_NOTICE,STATUS,VersionFirstReleased,VersionLastChanged\n0,100002-5,Specimen care is maintained,Find,Pt,^Patient,Ord,,SURVEY.PNDS,4,Specimen care is maintained,,,ACTIVE,2.72,2.72\n1,100003-3,Team communication is maintained throughout care,Find,Pt,^Patient,Ord,,SURVEY.PNDS,4,Team communication is maintained throughout care,,,ACTIVE,2.72,2.72\n2,100004-1,Demonstrates knowledge of the expected psychosocial responses to the procedure,Find,Pt,^Patient,Ord,,SURVEY.PNDS,4,Demonstrates knowledge of the expected psychosocial responses to the procedure,,,ACTIVE,2.72,2.72\n3,100005-8,Demonstrates knowledge of nutritional management related to the procedure,Find,Pt,^Patient,Ord,,SURVEY.PNDS,4,Demonstrates knowledge of nutritional management related to the procedure,,,ACTIVE,2.72,2.72\n4,100006-6,Demonstrates knowledge of medication management,Find,Pt,^Patient,Ord,,SURV

In [293]:
df_trim

,LOINC_NUM,LONG_COMMON_NAME
0,100002-5,Specimen care is maintained
1,100003-3,Team communication is maintained throughout care
2,100004-1,Demonstrates knowledge of the expected psychos...
3,100005-8,Demonstrates knowledge of nutritional manageme...
4,100006-6,Demonstrates knowledge of medication management
...,...,...
99074,99994-6,"Fluid, electrolyte, and acid-base balances are..."
99075,99995-3,Respiratory status is maintained at or improve...
99076,99996-1,Cardiovascular status is maintained at or impr...
99077,99997-9,Demonstrates AndOr reports adequate pain control


In [267]:
full_dataset_system_prompt = """
You are an assisstant for matching terminology between health record systems. 
You will be given a source text from one health system, which might be in another language.
You should output its equivalent name and code in LOINC clinical terminology.

Here is the LOINC dataset for context:
{dataset}
"""

full_dataset_user_prompt = """
Source text to output a LOINC term for: "{input_text}"
"""

In [295]:
# df_trim = df[["LOINC_NUM", "LONG_COMMON_NAME"]]
df_trim = df[["LOINC_NUM", "COMPONENT"]]

In [310]:
# for quick experimentation, remove a chunk of rows - if looks promising we can remove by nicheness instead
df_trim_2 = df_trim[50000:]

In [311]:
# use LLM to get input text CT synonym then search vector store. Also add hybrid search.

def long_prompt_mapping(input_query):
    # input_query = "malaria_RDT_result"

    full_dataset_user_prompt_formatted = full_dataset_user_prompt.format(input_text=input_query)
    message = client.messages.create(
        model= "claude-3-5-sonnet-20241022", #"claude-3-5-sonnet-20241022", #claude-3-5-haiku-20241022
        max_tokens=500,
        temperature=0,
        system=full_dataset_system_prompt.format(dataset=df_trim_2.to_csv()),
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": full_dataset_user_prompt_formatted
                    }
                ]
            }
        ]
    )
    synonym_expand_answer = message.content[0].text
    # print(f"LLM guess for the CT: {synonym_expand_answer}")
    # synonym_expand_retreived_data = loinc_store.search(input_query, search_kwargs={"k": 100})

    # # add keyword search to conduct hybrid search
    # keyword_search_results = df[df["LONG_COMMON_NAME"].str.contains(synonym_expand_answer)].LONG_COMMON_NAME.to_list()
    # keyword_search_results = [SearchResult(text=json.dumps({"LONG_COMMON_NAME": s}), metadata={}, score=None) for s in keyword_search_results]
    # search_results_combined = keyword_search_results+synonym_expand_retreived_data

    # # temp fix for missing loinc numbers
    # search_results_combined = [SearchResult(text=r.text, metadata={**r.metadata, 'LOINC_NUMBER': loinc_num_dict.get(json.loads(r.text)['LONG_COMMON_NAME'])}, score=r.score) for r in search_results_combined]

    # # tried this rule-based retreival, but there are unexpected issues e.g. "Body height" hits "Body height Mother"
    # # final_search_result = ""
    # # for search_result in synonym_expand_retreived_data:
    # #     if synonym_expand_answer in search_result.metadata["LONG_COMMON_NAME"]:
    # #         final_search_result = (search_result, loinc_num_dict[search_result.metadata["LONG_COMMON_NAME"]])
    # # final_search_result

    # synonym_expand_select_user_prompt_formatted = synonym_expand_select_user_prompt.format(input_text=input_query, LLM_guess=synonym_expand_answer, retreived_texts=search_results_combined)
    # message = client.messages.create(
    #     model= "claude-3-5-sonnet-20241022", #"claude-3-5-sonnet-20241022", #claude-3-5-haiku-20241022
    #     max_tokens=500,
    #     temperature=0,
    #     system=synonym_expand_select_system_prompt,
    #     messages=[
    #         {
    #             "role": "user",
    #             "content": [
    #                 {
    #                     "type": "text",
    #                     "text": synonym_expand_select_user_prompt_formatted
    #                 }
    #             ]
    #         }
    #     ]
    # )
    # synonym_expand_select_answer = message.content[0].text
    
    return synonym_expand_answer

In [312]:
results = []
for field, gold_label in zip(fields, gold_labels):
    result = long_prompt_mapping(field)
    results.append([field, gold_label, result])
results_prompt_df = pd.DataFrame(results, columns=["Input field", "Correct answer", "System answer"])
results_prompt_df.to_csv("results_long_prompt_1220.csv", index=False)

BadRequestError: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'prompt is too long: 201199 tokens > 200000 maximum'}}

Need to test more inputs to understand pros and cons of each approach.

In [ ]:
snomed_synonym_inputs = [
    "alert",
    "responsive_to_verbal_stimulus",
    "responsive_to_pain",
    "unresponsive",
    "negative",
    "positive",
    "current_smoker",
    "former_smoker",
    "never_smoked",
    "zero",
    "one_to_three",
    "four-to-seven",
    "eight_or_more",
    "normal",
    "abnormal",
    "needs_repeated",
    "glucose",
    "leucocytes",
    "nitrites",
    "ketones",
    "protein",
    "blood",
    "bilirubin",
    "normal",
    "konsultasi_dokter_gigi",
    "cabut_gigi_dengan_anestesi_(spuit)",
    "cabut_gigi_tanpa_anestesi_(anak)",
    "tambahan_per_gigi_cabut",
    "tambalan_gic_aduk",
    "premedikasi_tambalan_sementara_(eugenol/_devitalisasi)",
    "insisi_abses",
    "tambahan_injeksi_pehacaine_*utk_bpjs_tidak_ada_biaya",
    "condoms",
    "combined_pill",
    "progesterone_only_pill",
    "depo_provera_injection_im",
    "implant",
    "copper_iud_coil",
    "hormonal_iud_morena",
    "vasectomy",
    "tubectomy",
    "current_drug_user",
    "former_drug_user",
    "never_a_drug_user"
]

Pipeline for searching for a body site (NOT synonym)

Searching for the correct body site for a diagnosis or process presents a different kind of problem from searching for a similar term (the mapping issue above). We cannot use RAG directly, because the input an output will not be "similar".

- - -

The tested method will use the following steps to try and generate a body site:

Step 1: Use an LLM to directly guess the correct SNOMED body site. From previous experiments with ChatGPT, we found that the clinical term is often accurate, but the code will be hallucinated.

Step 2: Use the LLM output from step 1 to search the SNOMED terms using a similarity search (remove digits from the answer; filter for body site). Take the top 1 match. The code field in the metadata is our SNOMED body site code output. We use 

- - -
Problem: There are a lot of (reasonable) errors in the output. Not sure if the answers require contextual knowledge of the conventions of the specific health record system in use, or medical knowledge. Or, are there many correct outputs in some cases?

Errors found:

migraine -> head (nervous system)
constipation -> large intestine (intestinal system)
hypokalaemia -> Blood (Structure of cardiovascular system)
Depressive_episode,_unspecified -> Brain structure (Central nervous system)  [note that e.g. bipolar and schizophrenia are brain structure]

In [ ]:
body_site_step_1_system_prompt = """
You are an assistant for tagging diagnoses and processes in health records with
the correct body sites as defined in SNOMED Clinical Terminology.
You will be given an input text from a health record. 
Simply state the SNOMED body site term for the input.
"""
# 
body_site_step_1_user_prompt = """
Input to return a body site for: "{input_text}"
"""


In [ ]:
# "You should return the
# correct SNOMED term for the input, or several options if the condition involves
# several parts of the body.""

In [353]:
def get_snomed_bodysite(input_query):

    body_site_step_1_user_prompt_formatted = body_site_step_1_user_prompt.format(input_text=input_query.replace("_", " "))

    message = client.messages.create(
        model= "claude-3-5-sonnet-20241022", #"claude-3-5-sonnet-20241022", #"claude-3-5-haiku-20241022"
        max_tokens=500,
        temperature=0,
        system=body_site_step_1_system_prompt,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": body_site_step_1_user_prompt_formatted
                    }
                ]
            }
        ]
    )
    body_site_answer = message.content[0].text
    body_site_snomed_entry = snomed_store.search(body_site_answer, search_kwargs={"k": 1})
    found_name = body_site_snomed_entry[0].metadata["Description"]
    found_code = body_site_snomed_entry[0].metadata["Code"]

    return f"llm guess: {body_site_answer}; matched entry: {found_name};{found_code}"


In [344]:
get_snomed_bodysite("typhoid fever")

'llm guess: Structure of entire gastrointestinal tract; matched entry: Gastrointestinal tract structure (body structure);122865005.0'

In [349]:
snomed_bodysite_inputs_trim = [
    "Typhoid fever",
    "Dysentery, bacterial",
    "Other bacterial intestinal infections",
    "Amoebiasis",
    "Diarrhoea and gastroenteritis of presumed infectious origin",
    "Respiratory tuberculosis, not confirmed bacteriologically or histologically",
    "Tuberculosis of other organs",
    "Leprosy (Hansen's Disease)",
    "Other tetanus",
    "Whooping cough",
    "Septicaemia, unspecified",
    "Unspecified sexually transmitted disease",
    "Trachoma",
    "Rabies",
    "Meningitis, Viral",
    "Dengue fever",
    "Varicella (chickenpox)",
    "Herpes Zoster",
    "Measles",
    "Molluscum contagiosum",
    "Acute hepatitis A",
    "Unspecified Human Immunodeficiency Virus (HIV) Disease",
    "Mumps",
    "Viral infection of unspecified site",
    "Candidiasis, unspecified",
    "Plasmodium falciparum malaria",
    "Unspecified malaria",
    "Schistosomiasis, unspecified",
    "Filariasis",
    "Ascariasis",
    "Unspecified intestinal parasitism",
    "Helminthiasis, unspecified",
    "Scabies",
    "Sequelae of tuberculosis",
    "Lipoma",
    "Leiomyoma of uterus",
    "Benign neoplasm of other and unspecified sites",
    "Thalassaemia, unspecified",
    "Anaemia, unspecified",
    "Other hypothyroidism",
    "Thyrotoxicosis with diffuse goitre (Graves' disease)",
    "Thyrotoxicosis with toxic single thyroid nodule",
    "Thyrotoxicosis, unspecified",
    "Insulin-dependent diabetes mellitus",
    "Non-insulin-dependent diabetes mellitus",
    "Polycystic ovarian syndrome",
    "Unspecified protein-energy malnutrition",
    "Obesity, unspecified",
    "Hyperlipidaemia, unspecified",
    "Volume depletion",
    "Hypokalaemia",
    "Substance abuse",
    "Schizophrenia",
    "Acute and transient psychotic disorder, unspecified",
    "Bipolar disorder",
    "Depressive episode, unspecified",
    "Generalized anxiety disorder",
    "Emotional or mental trauma; reaction to severe stress; adjustment disorders",
    "Post traumatic stress disorder, unspecified",
    "Eating disorder,unspecified",
    "Postpartum depression",
    "Neurodevelopmental disorders",
    "Meningitis, unspecified",
    "Parkinson's Disease",
    "Epilepsy",
    "Epilepsy, unspecified"
]

In [351]:
len(snomed_bodysite_inputs_trim)

66

In [ ]:
results = []
for field in snomed_bodysite_inputs_trim:
    result = get_snomed_bodysite(field)
    results.append([field, result])
snomed_bodysite_df = pd.DataFrame(results, columns=["Input field", "System answer"])
snomed_bodysite_df.to_csv("results_1220_snomed_bodysite.csv", index=False)

In [359]:
snomed_bodysite_df

,Input field,System answer
0,Typhoid fever,llm guess: No specific body site is directly a...
1,"Dysentery, bacterial",llm guess: Structure of entire gastrointestina...
2,Other bacterial intestinal infections,llm guess: Body structure: Intestinal structur...
3,Amoebiasis,llm guess: Intestinal structure (body structur...
4,Diarrhoea and gastroenteritis of presumed infe...,llm guess: Body site: Gastrointestinal tract s...
...,...,...
61,Neurodevelopmental disorders,llm guess: Brain structure (body structure); m...
62,"Meningitis, unspecified",llm guess: Meninges structure (body structure)...
63,Parkinson's Disease,llm guess: Brain structure (body structure); m...
64,Epilepsy,llm guess: Entire brain (body structure); matc...


In [358]:
# results_format = [[f'"""{r[0]}"""', r[1]] for r in results]
snomed_bodysite_format_df = pd.DataFrame(results_format, columns=["Input field", "System answer"])
snomed_bodysite_format_df

,Input field,System answer
0,"""""""Typhoid fever""""""",llm guess: No specific body site is directly a...
1,"""""""Dysentery, bacterial""""""",llm guess: Structure of entire gastrointestina...
2,"""""""Other bacterial intestinal infections""""""",llm guess: Body structure: Intestinal structur...
3,"""""""Amoebiasis""""""",llm guess: Intestinal structure (body structur...
4,"""""""Diarrhoea and gastroenteritis of presumed i...",llm guess: Body site: Gastrointestinal tract s...
...,...,...
61,"""""""Neurodevelopmental disorders""""""",llm guess: Brain structure (body structure); m...
62,"""""""Meningitis, unspecified""""""",llm guess: Meninges structure (body structure)...
63,"""""""Parkinson's Disease""""""",llm guess: Brain structure (body structure); m...
64,"""""""Epilepsy""""""",llm guess: Entire brain (body structure); matc...


In [360]:
snomed_bodysite_format_df.to_csv("results_1220_snomed_bodysite_formatted.csv", index=False)

In [ ]:
# TODO add filtering for body site

Examine the data

In [54]:
import pandas as pd

In [59]:
df = pd.read_csv("/Users/hanna/openfn/ai_experiments/data/SNOMED-CT-Code-Value-Semantic-Set/SNOMED-CT-Code-Value-Semantic-Set.csv")
df_body = df[df["Value Set Name"]=="Body Site Value Set"] 
df_body.to_csv("/Users/hanna/openfn/ai_experiments/data/SNOMED-CT-Code-Value-Semantic-Set/SNOMED-CT-Code-Value-Semantic-Set_body_site.csv", index=False)

In [60]:
df_body[df_body["Description"].str.contains("blood")]

,Code System,Value Set Name,Code,Description,Purpose: Clinical Focus,Value Set OID,Code System OID
281,SNOMEDCT,Body Site Value Set,91747007,Structure of lumen of blood vessel (body struc...,"All SNOMED CT anatomic structures, locations, ...",2.16.840.1.113883.3.88.12.3221.8.9,2.16.840.1.113883.6.96
755,SNOMEDCT,Body Site Value Set,89615005,Abnormal cellular component of blood (morpholo...,"All SNOMED CT anatomic structures, locations, ...",2.16.840.1.113883.3.88.12.3221.8.9,2.16.840.1.113883.6.96
1142,SNOMEDCT,Body Site Value Set,871850003,Entire blood vessel within duodenum (body stru...,"All SNOMED CT anatomic structures, locations, ...",2.16.840.1.113883.3.88.12.3221.8.9,2.16.840.1.113883.6.96
1143,SNOMEDCT,Body Site Value Set,871851004,Structure of blood vessel within esophagus (bo...,"All SNOMED CT anatomic structures, locations, ...",2.16.840.1.113883.3.88.12.3221.8.9,2.16.840.1.113883.6.96
1144,SNOMEDCT,Body Site Value Set,871852006,Entire blood vessel within esophagus (body str...,"All SNOMED CT anatomic structures, locations, ...",2.16.840.1.113883.3.88.12.3221.8.9,2.16.840.1.113883.6.96
...,...,...,...,...,...,...,...
38983,SNOMEDCT,Body Site Value Set,1149178005,Entire blood vessel in colonic submucosa and c...,"All SNOMED CT anatomic structures, locations, ...",2.16.840.1.113883.3.88.12.3221.8.9,2.16.840.1.113883.6.96
38984,SNOMEDCT,Body Site Value Set,1149179002,Structure of lymphatic vessel and/or small blo...,"All SNOMED CT anatomic structures, locations, ...",2.16.840.1.113883.3.88.12.3221.8.9,2.16.840.1.113883.6.96
38985,SNOMEDCT,Body Site Value Set,1149181000,Entire lymphatic vessel and small blood vessel...,"All SNOMED CT anatomic structures, locations, ...",2.16.840.1.113883.3.88.12.3221.8.9,2.16.840.1.113883.6.96
39464,SNOMEDCT,Body Site Value Set,111014003,Dual red blood cell population (morphologic ab...,"All SNOMED CT anatomic structures, locations, ...",2.16.840.1.113883.3.88.12.3221.8.9,2.16.840.1.113883.6.96


# You can test the mapping functions below this cell by changing the text in input_query and running the cell.

In [ ]:
# Synonyms – mapping an input to LOINC terms
# Method 1 - Get search results, and have LLM pick from the retreived terms
input_query = "hepatitis b result"
synonym_method_1(input_query)

'Since the source text only specifies "hepatitis b result" without any additional details about specific antigens, antibodies, or test methods, I\'ll select the most general Hepatitis B test from the options.\n\nI recommend:\n{"LONG_COMMON_NAME": "Hepatitis B virus Ab [Interpretation] in Serum or Plasma", "METHOD_TYP": "", "CLASS": "MICRO", "SYSTEM": "Ser/Plas"}\n\nRationale:\n1. This is the most generic Hepatitis B test option available\n2. It\'s an interpretation test that can encompass multiple markers\n3. It doesn\'t specify a particular antigen or antibody type\n4. It allows for both serum and plasma specimens\n5. It doesn\'t specify a particular testing methodology, keeping it flexible\n\nThe other options are more specific (testing for particular antigens or antibodies) or use specific methodologies, while the source text doesn\'t provide that level of detail.'

In [ ]:
# Synonyms – mapping an input to LOINC terms
# Method 2 – LLM outputs what it thinks the term is, then we use it to retreive results using hybrid search (similarity search + keyword search).
# We then use the LLM again to pick the correct exact term and the LOINC code.
input_query = "hepatitis b result"
synonym_method_2(input_query)

LLM guess for the CT: Hepatitis B virus surface Ag [Presence] in Serum


'Hepatitis B virus surface Ag [Presence] in Serum or Plasma by Immunoassay;5196-1'

In [ ]:
# Snomed body site mapping
# Method 1 – LLM outputs what it thinks the term is, then we use it to retreive results.
# Note: Currently the SNOMED code retreival part fails - the retreival step needs fixing.

input_query = "epilepsy"
get_snomed_bodysite(input_query)

'llm guess: For "epilepsy", the relevant SNOMED body site is:\n\nBrain structure (entire brain); matched entry: Entire ependyma of brain ventricle (body structure);731442007.0'